In [6]:
input_file = open("untitled.txt", 'r')

seq1 = input_file.readline().replace('\n', '')
seq2 = input_file.readline().replace('\n', '')

aa_no = {'A':0,'C':1,'D':2,'E':3,'F':4,'G':5,'H':6,'I':7,'K':8,'L':9,'M':10,'N':11,'P':12,'Q':13,'R':14,'S':15,'T':16,'V':17,'W':18,'Y':19}

score_BLOSUM = [[4,0,-2,-1,-2,0,-2,-1,-1,-1,-1,-2,-1,-1,-1,1,0,0,-3,-2],
                [0,9,-3,-4,-2,-3,-3,-1,-3,-1,-1,-3,-3,-3,-3,-1,-1,-1,-2,-2],
                [-2,-3,6,2,-3,-1,-1,-3,-1,-4,-3,1,-1,0,-2,0,-1,-3,-4,-3],
                [-1,-4,2,5,-3,-2,0,-3,1,-3,-2,0,-1,2,0,0,-1,-2,-3,-2],
                [-2,-2,-3,-3,6,-3,-1,0,-3,0,0,-3,-4,-3,-3,-2,-2,-1,1,3],
                [0,-3,-1,-2,-3,6,-2,-4,-2,-4,-3,0,-2,-2,-2,0,-2,-3,-2,-3],
                [-2,-3,-1,0,-1,-2,8,-3,-1,-3,-2,1,-2,0,0,-1,-2,-3,-2,2],
                [-1,-1,-3,-3,0,-4,-3,4,-3,2,1,-3,-3,-3,-3,-2,-1,3,-3,-1],
                [-1,-3,-1,1,-3,-2,-1,-3,5,-2,-1,0,-1,1,2,0,-1,-2,-3,-2],
                [-1,-1,-4,-3,0,-4,-3,2,-2,4,2,-3,-3,-2,-2,-2,-1,1,-2,-1],
                [-1,-1,-3,-2,0,-3,-2,1,-1,2,5,-2,-2,0,-1,-1,-1,1,-1,-1],
                [-2,-3,1,0,-3,0,1,-3,0,-3,-2,6,-2,0,0,1,0,-3,-4,-2],
                [-1,-3,-1,-1,-4,-2,-2,-3,-1,-3,-2,-2,7,-1,-2,-1,-1,-2,-4,-3],
                [-1,-3,0,2,-3,-2,0,-3,1,-2,0,0,-1,5,1,0,-1,-2,-2,-1],
                [-1,-3,-2,0,-3,-2,0,-3,2,-2,-1,0,-2,1,5,-1,-1,-3,-3,-2],
                [1,-1,0,0,-2,0,-1,-2,0,-2,-1,1,-1,0,-1,4,1,-2,-3,-2],
                [0,-1,-1,-1,-2,-2,-2,-1,-1,-1,-1,0,-1,-1,-1,1,5,0,-2,-2],
                [0,-1,-3,-2,-1,-3,-3,3,-2,1,1,-3,-2,-2,-3,-2,0,4,-3,-1],
                [-3,-2,-4,-3,1,-2,-2,-3,-3,-2,-1,-4,-4,-2,-3,-3,-2,-3,11,2],
                [-2,-2,-3,-2,3,-3,2,-1,-2,-1,-1,-2,-3,-1,-2,-2,-2,-1,2,7]]


def getscore(AA1,AA2) : return score_BLOSUM[aa_no[AA1]][aa_no[AA2]]

sigma = -11
epsilon = -1

def global_alignment_affine_gaps(s1,s2,sigma,epsilon):

    n = len(s1)
    m = len(s2)
    
    s = [[[0 for i in xrange(m+1)] for j in range(n+1)] for k in xrange(3)]
    backtrack = [[[0 for i in xrange(m+1)] for j in range(n+1)] for k in xrange(3)]

    for i in range(1,n+1):
        s[0][i][0] = sigma + (i-1)*epsilon
        s[1][i][0] = sigma + (i-1)*epsilon
        s[2][i][0] = 10*sigma

    for j in range(1,m+1):
        s[0][0][j] = 10*sigma
        s[1][0][j] = sigma + (j-1)*epsilon
        s[2][0][j] = sigma + (j-1)*epsilon

    for i in range(1,n+1):

        for j in range(1,m+1):

            low_scores = [s[0][i-1][j] + epsilon,s[1][i-1][j] + sigma]
            
            s[0][i][j] = max(low_scores)
            backtrack[0][i][j] = low_scores.index(s[0][i][j])
            
            up_scores = [s[2][i][j-1] + epsilon,s[1][i][j-1] + sigma]
            s[2][i][j] = max(up_scores)
            backtrack[2][i][j] = up_scores.index(s[2][i][j])

            mid_scores = [s[0][i][j],s[1][i-1][j-1] + getscore(s1[i-1],s2[j-1]),s[2][i][j]]
            s[1][i][j] = max(mid_scores)
            backtrack[1][i][j] = mid_scores.index(s[1][i][j])

    max_scores = [s[0][i][j],s[1][i][j],s[2][i][j]]
    max_score = max(max_scores)
    backtrack_matrix = max_scores.index(max_score)

    insert_indel = lambda word, i: word[:i] + '-' + word[i:]


    s1_allign = s1
    s2_allign = s2

    while i*j != 0:
        if backtrack_matrix == 0:
            if backtrack[0][i][j] == 1:
                backtrack_matrix = 1
            i -= 1
            s2_allign  = insert_indel(s2_allign,j)
        elif backtrack_matrix == 1:
            if backtrack[1][i][j] == 0:
                backtrack_matrix = 0

            elif backtrack[1][i][j] == 2:

                backtrack_matrix = 2

            else:
                i -= 1
                j -= 1

        else:
            if backtrack[2][i][j] == 1:
                backtrack_matrix = 1
            j -= 1
            s1_allign = insert_indel(s1_allign,i)

    for _ in xrange(i):
        s2_allign = insert_indel(s2_allign,0)

    for _ in xrange(j):
        s1_allign = insert_indel(s1_allign,0) 
        
    return max_score,s1_allign,s2_allign

max_score, s1, s2 = global_alignment_affine_gaps(seq1, seq2, sigma, epsilon)

print(max_score)
print(s1)
print(s2)

261
PPWPTKVWSKSKWYWECAAPTAHQSKMPTWDEVAVIAVITHNMSYPCY--------CMVRFDAEPQEVNKLAGSDGSVG--TTLVGPYLNQ
PPS-----SKSKWYWECIAP---QSKMPTWTECAVIAVITHNMSYPCTPDRMQTPGMQVRFDAEPQE----AGSDGSVGLCMTLVGPYLNQ


In [10]:
input_file = open("untitled.txt", 'r')

seq1 = input_file.readline().replace('\n', '')
seq2 = input_file.readline().replace('\n', '')

aa_no = {'A':0,'C':1,'D':2,'E':3,'F':4,'G':5,'H':6,'I':7,'K':8,'L':9,'M':10,'N':11,'P':12,'Q':13,'R':14,'S':15,'T':16,'V':17,'W':18,'Y':19}

score_BLOSUM = [[4,0,-2,-1,-2,0,-2,-1,-1,-1,-1,-2,-1,-1,-1,1,0,0,-3,-2],
                [0,9,-3,-4,-2,-3,-3,-1,-3,-1,-1,-3,-3,-3,-3,-1,-1,-1,-2,-2],
                [-2,-3,6,2,-3,-1,-1,-3,-1,-4,-3,1,-1,0,-2,0,-1,-3,-4,-3],
                [-1,-4,2,5,-3,-2,0,-3,1,-3,-2,0,-1,2,0,0,-1,-2,-3,-2],
                [-2,-2,-3,-3,6,-3,-1,0,-3,0,0,-3,-4,-3,-3,-2,-2,-1,1,3],
                [0,-3,-1,-2,-3,6,-2,-4,-2,-4,-3,0,-2,-2,-2,0,-2,-3,-2,-3],
                [-2,-3,-1,0,-1,-2,8,-3,-1,-3,-2,1,-2,0,0,-1,-2,-3,-2,2],
                [-1,-1,-3,-3,0,-4,-3,4,-3,2,1,-3,-3,-3,-3,-2,-1,3,-3,-1],
                [-1,-3,-1,1,-3,-2,-1,-3,5,-2,-1,0,-1,1,2,0,-1,-2,-3,-2],
                [-1,-1,-4,-3,0,-4,-3,2,-2,4,2,-3,-3,-2,-2,-2,-1,1,-2,-1],
                [-1,-1,-3,-2,0,-3,-2,1,-1,2,5,-2,-2,0,-1,-1,-1,1,-1,-1],
                [-2,-3,1,0,-3,0,1,-3,0,-3,-2,6,-2,0,0,1,0,-3,-4,-2],
                [-1,-3,-1,-1,-4,-2,-2,-3,-1,-3,-2,-2,7,-1,-2,-1,-1,-2,-4,-3],
                [-1,-3,0,2,-3,-2,0,-3,1,-2,0,0,-1,5,1,0,-1,-2,-2,-1],
                [-1,-3,-2,0,-3,-2,0,-3,2,-2,-1,0,-2,1,5,-1,-1,-3,-3,-2],
                [1,-1,0,0,-2,0,-1,-2,0,-2,-1,1,-1,0,-1,4,1,-2,-3,-2],
                [0,-1,-1,-1,-2,-2,-2,-1,-1,-1,-1,0,-1,-1,-1,1,5,0,-2,-2],
                [0,-1,-3,-2,-1,-3,-3,3,-2,1,1,-3,-2,-2,-3,-2,0,4,-3,-1],
                [-3,-2,-4,-3,1,-2,-2,-3,-3,-2,-1,-4,-4,-2,-3,-3,-2,-3,11,2],
                [-2,-2,-3,-2,3,-3,2,-1,-2,-1,-1,-2,-3,-1,-2,-2,-2,-1,2,7]]


def getscore(AA1,AA2) : return score_BLOSUM[aa_no[AA1]][aa_no[AA2]]

sigma = -5

def middle_col_score(s1,s2,sigma):
    n = len(s1)
    m = len(s2)
    s = [[i*j*(-sigma) for j in xrange(-1,1)] for i in xrange(n+1)]
    s[0][1] = sigma
    backtrack = [0]*(n+1)

    for j in xrange(1,m/2 + 1):
        for i in xrange(0,n+1):
            if i == 0:
                s[i][1] = j*sigma
            else:
                scores = [s[i-1][0] + getscore(s1[i-1],s2[j-1]),s[i][0] + sigma, s[i-1][1] + sigma]
                s[i][1] = max(scores)
                backtrack[i] = scores.index(s[i][1])

        if j != m/2 : s = [[row[1]]*2 for row in s]

    return [row[1] for row in s], backtrack



def middle_edge(s1,s2,sigma):

    source_to_middle = middle_col_score(seq1, seq2, sigma)[0]
    middle_to_sink, backtrack = map(lambda l: l[::-1], middle_col_score(seq1[::-1], seq2[::-1]+['', '$'][len(seq2) % 2 == 1 and len(seq2) > 1], sigma))
    scores = map(sum, zip(source_to_middle, middle_to_sink))
    max_middle = max(xrange(len(scores)), key=lambda i: scores[i])

    if max_middle == len(scores) - 1 : next_node = (max_middle, len(seq2)/2 + 1)
    else : next_node = [(max_middle + 1, len(seq2)/2 + 1), (max_middle, len(seq2)/2 + 1), (max_middle + 1, len(seq2)/2),][backtrack[max_middle]]

    return (max_middle, len(seq2)/2), next_node

middle_edge(seq1, seq2, sigma)

((505, 523), (506, 524))

In [15]:
input_file = open("untitled.txt", 'r')

seq1 = input_file.readline().replace('\n', '')
seq2 = input_file.readline().replace('\n', '')

aa_no = {'A':0,'C':1,'D':2,'E':3,'F':4,'G':5,'H':6,'I':7,'K':8,'L':9,'M':10,'N':11,'P':12,'Q':13,'R':14,'S':15,'T':16,'V':17,'W':18,'Y':19}

score_BLOSUM = [[4,0,-2,-1,-2,0,-2,-1,-1,-1,-1,-2,-1,-1,-1,1,0,0,-3,-2],
                [0,9,-3,-4,-2,-3,-3,-1,-3,-1,-1,-3,-3,-3,-3,-1,-1,-1,-2,-2],
                [-2,-3,6,2,-3,-1,-1,-3,-1,-4,-3,1,-1,0,-2,0,-1,-3,-4,-3],
                [-1,-4,2,5,-3,-2,0,-3,1,-3,-2,0,-1,2,0,0,-1,-2,-3,-2],
                [-2,-2,-3,-3,6,-3,-1,0,-3,0,0,-3,-4,-3,-3,-2,-2,-1,1,3],
                [0,-3,-1,-2,-3,6,-2,-4,-2,-4,-3,0,-2,-2,-2,0,-2,-3,-2,-3],
                [-2,-3,-1,0,-1,-2,8,-3,-1,-3,-2,1,-2,0,0,-1,-2,-3,-2,2],
                [-1,-1,-3,-3,0,-4,-3,4,-3,2,1,-3,-3,-3,-3,-2,-1,3,-3,-1],
                [-1,-3,-1,1,-3,-2,-1,-3,5,-2,-1,0,-1,1,2,0,-1,-2,-3,-2],
                [-1,-1,-4,-3,0,-4,-3,2,-2,4,2,-3,-3,-2,-2,-2,-1,1,-2,-1],
                [-1,-1,-3,-2,0,-3,-2,1,-1,2,5,-2,-2,0,-1,-1,-1,1,-1,-1],
                [-2,-3,1,0,-3,0,1,-3,0,-3,-2,6,-2,0,0,1,0,-3,-4,-2],
                [-1,-3,-1,-1,-4,-2,-2,-3,-1,-3,-2,-2,7,-1,-2,-1,-1,-2,-4,-3],
                [-1,-3,0,2,-3,-2,0,-3,1,-2,0,0,-1,5,1,0,-1,-2,-2,-1],
                [-1,-3,-2,0,-3,-2,0,-3,2,-2,-1,0,-2,1,5,-1,-1,-3,-3,-2],
                [1,-1,0,0,-2,0,-1,-2,0,-2,-1,1,-1,0,-1,4,1,-2,-3,-2],
                [0,-1,-1,-1,-2,-2,-2,-1,-1,-1,-1,0,-1,-1,-1,1,5,0,-2,-2],
                [0,-1,-3,-2,-1,-3,-3,3,-2,1,1,-3,-2,-2,-3,-2,0,4,-3,-1],
                [-3,-2,-4,-3,1,-2,-2,-3,-3,-2,-1,-4,-4,-2,-3,-3,-2,-3,11,2],
                [-2,-2,-3,-2,3,-3,2,-1,-2,-1,-1,-2,-3,-1,-2,-2,-2,-1,2,7]]


def getscore(AA1,AA2) : return score_BLOSUM[aa_no[AA1]][aa_no[AA2]]

sigma = -5

def global_allignment(s1,s2,indel_penalty):

    n = len(s1)
    m = len(s2)
    s = [[0 for repeat_j in xrange(len(s2)+1)] for repeat_i in xrange(len(s1)+1)]
    backtrack = [['0' for repeat_j in xrange(len(s2)+1)] for repeat_i in xrange(len(s1)+1)]
    for i in xrange(1,n+1):
        s[i][0] = s[i-1][0] + indel_penalty
        backtrack[i][0] = 'U'
    for j in xrange(1,m+1):
        s[0][j] = s[0][j-1] + indel_penalty
        backtrack[0][j] = 'L'
    for i in xrange(1,n+1):
        for j in xrange(1,m+1):
            up_score = s[i-1][j] + indel_penalty
            left_score = s[i][j-1] + indel_penalty
            diag_score = s[i-1][j-1] + getscore(s1[i-1],s2[j-1])
            s[i][j] = max(left_score,up_score,diag_score)
            if diag_score >= up_score:
                if diag_score >= left_score:
                    backtrack[i][j] = 'D'
                elif diag_score < left_score:
                    backtrack[i][j] = 'L'
            elif diag_score < up_score:
                if up_score >= left_score:
                    backtrack[i][j] = 'U'
                else:
                    backtrack[i][j] = 'L'
    return s[n][m],backtrack



def backtrack_allignment(seq1,seq2,indel):

    backtrack = global_allignment(seq1,seq2,indel)[1]
    insert_indel = lambda word, i: word[:i] + '-' + word[i:]
    seq1_aligned, seq2_aligned = seq1, seq2
    i, j = len(seq1), len(seq2)
    while 1:
        if backtrack[i][j] == '0':
            break
        if backtrack[i][j] == 'U':
            i -= 1
            seq2_aligned = insert_indel(seq2_aligned, j)
        elif backtrack[i][j] == 'L':
            j -= 1
            seq1_aligned = insert_indel(seq1_aligned, i)
        else:
            i -= 1
            j -= 1
    for repeat in xrange(i):
        seq2_aligned = insert_indel(seq2_aligned, 0)
    for repeat in xrange(j):
        seq1_aligned = insert_indel(seq1_aligned, 0)
    return global_allignment(seq1,seq2,indel)[0],seq1_aligned,seq2_aligned

def space_efficient_global_alignment(s1, s2, sigma):

    def linear_space_alignment(top, bottom, left, right):

        if left == right:
            return [s1[top:bottom], '-'*(bottom-top)]

        elif top == bottom:
            return ['-'*(right-left), s2[left:right]]

        elif bottom - top == 1 or left - right == 1:
            return backtrack_alignment(s1[top:bottom], s2[left:right], sigma)

        else:
            mid_node, next_node = middle_edge(s1[top:bottom], s2[left:right], sigma)
            mid_node = tuple(map(sum, zip(mid_node, [top, left])))
            next_node = tuple(map(sum, zip(next_node, [top, left])))
            current = [['-', s1[mid_node[0] % len(s1)]][next_node[0] - mid_node[0]], ['-', s2[mid_node[1] % len(s2)]][next_node[1] - mid_node[1]]]
            a = linear_space_alignment(top, mid_node[0], left, mid_node[1])
            b = linear_space_alignment(next_node[0], bottom, next_node[1], right)
            return [a[i] + current[1] + b[i] for i in xrange(2)]
        
    s1_allign, s2_allign = linear_space_alignment(0,len(s1),0,len(s2))
    score = sum([sigma if '-' in pair else getscore(pair[0],pair[1]) for pair in zip(s1_allign,s2_allign)])
    
    return str(score), s1_allign, s2_allign

print(space_efficient_global_alignment(seq1, seq2, sigma))

RuntimeError: maximum recursion depth exceeded while calling a Python object

In [75]:
import numpy as np
import operator

input_file = open("untitled.txt", 'r')

seq1 = input_file.readline().replace('\n', '')
seq2 = input_file.readline().replace('\n', '')
seq3 = input_file.readline().replace('\n', '')

def MultipleLongestCommonSequence(seq1, seq2, seq3) :
    
    n = len(seq1)
    m = len(seq2)
    o = len(seq3)
    
    s = np.zeros((n + 1, m + 1, o + 1), dtype = int)
    backtracking = np.zeros((n + 1, m + 1, o + 1), dtype = int)
    
    for i in range(1, n + 1) : 
        
        for j in range(1, m + 1) :
            
            for k in range(1, o + 1) :
                
                weight = 0
                if(seq1[i - 1] == seq2[j - 1] and seq2[j - 1] == seq3[k - 1]) :
                    weight = 1
                    
                possible_edge = [s[i - 1][j - 1][k - 1] + weight, s[i - 1][j][k], s[i][j - 1][k], s[i][j][k - 1], s[i - 1][j - 1][k], s[i - 1][j][k - 1], s[i][j - 1][k - 1]]
                backtracking[i][j][k], s[i][j][k] = max(enumerate(possible_edge), key = operator.itemgetter(1))        
    
    indel = lambda sequence, x : sequence[:x] + '-' + sequence[x:]
    
    while i * j * k != 0 :
        if(backtracking[i][j][k] == 0) : 
            i -= 1
            j -= 1
            k -= 1
            
        elif(backtracking[i][j][k] == 1) :
            seq2 = indel(seq2, j)
            seq3 = indel(seq3, k)
            i -= 1
            
        elif(backtracking[i][j][k] == 2) :
            seq1 = indel(seq1, i)
            seq3 = indel(seq3, k)
            j -= 1
            
        elif(backtracking[i][j][k] == 3) :
            seq1 = indel(seq1, i)
            seq2 = indel(seq2, j)
            k -= 1
            
        elif(backtracking[i][j][k] == 4) :
            seq3 = indel(seq3, k)
            i -= 1
            j -= 1
            
        elif(backtracking[i][j][k] == 5) :
            seq2 = indel(seq2, j)
            i -= 1
            k -= 1
            
        elif(backtracking[i][j][k] == 6) :
            seq1 = indel(seq1, i)
            j -= 1
            k -= 1
    
    length = max(len(seq1), len(seq2), len(seq3))
    
    while(len(seq1) != length) : seq1 = '-' + seq1
    while(len(seq2) != length) : seq2 = '-' + seq2
    while(len(seq3) != length) : seq3 = '-' + seq3
        
    return s[n][m][o], seq1, seq2, seq3
    
score, s1, s2, s3 = MultipleLongestCommonSequence(seq1, seq2, seq3)
print(score)
print(s1)
print(s2)
print(s3)

7
-CCAATA-C-GAC-
GCC-TTA-C-G-CT
-CC-CTAGCGG-C-
